In [ ]:
import threading
import time

In [ ]:
threading.enumerate()

## Потоки и присоединение к основному потоку
Создайте поток t, запустите его


**1) Запустите ячейку ниже и подождите несколько секунд, а после этого запустите следующую ячейке (с print и join)**

Почему так произошло?

**2) Запустите ячейку ниже и сразу же запустите ячейку с print и join.**

Результат разный? Почему?

In [ ]:
def non_daemon():
    time.sleep(5)
    print('Test non-daemon')


# создайте поток
t = threading.______(name='non-daemon', target=non_daemon)

# запустите поток
t._____()

# выведем на экран все потоки, основного потока
threading.enumerate()

In [ ]:
print('Test one')
t.join()
print('Test one')

In [ ]:
def non_daemon():
    time.sleep(5)
    print('Test non-daemon')


# создайте поток
t = threading.______(name='non-daemon', target=non_daemon)

# запустите поток
t._____()

# выведем на экран все потоки, основного потока
threading.enumerate()

In [ ]:
print('Test one')
print('Test one')
t.join()

## Многопоточность и ускорение расчётов
Реализуйте простое численное интегрирование и проверьте работу многопоточной версии


![Well](https://uploads.toptal.io/blog/image/126087/toptal-blog-image-1526311066247-4ce28d0e2a6878d80c5374d2c53e8aff.png)

In [ ]:
import math

In [ ]:
def integrate(f, a, b, *, n_iter=1000):
    acc = 0
    step = (b - a) / ______
    for i in range(n_iter):
        acc += f(a + i * step) * step
    return acc

In [ ]:
# проверьте её работу
integrate(math.cos, 0, math.pi / 2)

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import partial

In [ ]:
# теперь сделам то же самое с помощью пула потоков ThreadPoolExecutor

def integrate_async(f, a, b, *, n_jobs, n_iter=1000):
    executor = ThreadPoolExecutor(max_workers=n_jobs)
    # partial создаёт новую функцию, для которой часть параметров исходной зафиксированы
    spawn = partial(executor.submit, integrate, f, n_iter=n_iter // n_jobs)
    
    step = (b - a) / n_jobs
    fs = [spawn(a + i * step, a + (i + 1) * step) for i in range(n_jobs)]
    return sum(f.result() for f in as_completed(fs))

Что выйдет по времени и почему так?

In [ ]:
%%timeit -n100
integrate(math.cos, 0, math.pi / 2, n_iter=10**5)

In [ ]:
%%timeit -n100
integrate_async(math.cos, 0, math.pi / 2, n_iter=10**5, n_jobs=2)

In [ ]:
%%timeit -n100
integrate_async(math.cos, 0, math.pi / 2, n_iter=10**5, n_jobs=4)

## Многопроцессность

![Well](https://uploads.toptal.io/blog/image/126088/toptal-blog-image-1526311071925-a4dc10cda4cf6b88a7f11fd5d4c76f9a.png)

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed

In [ ]:
def integrate_async_multiproc(f, a, b, *, n_jobs, n_iter=1000):
    executor = ProcessPoolExecutor(max_workers=n_jobs)
    spawn = partial(executor.submit, integrate, f, n_iter=n_iter // n_jobs)
    
    step = (b - a) / n_jobs
    fs = [spawn(a + i * step, a + (i + 1) * step) for i in range(n_jobs)]
    return sum(f.result() for f in as_completed(fs))

In [ ]:
# если вы на Windows, поздравляю - код упадёт
%%timeit -n100
integrate_async_multiproc(math.cos, 0, math.pi / 2, n_iter=10**5, n_jobs=2)

## Многопроцессность и joblib

In [ ]:
from joblib import Parallel, delayed

In [ ]:
def integrate_async_multiproc(f, a, b, *, n_jobs, n_iter=1000, backend=None):
    step = (b - a) / n_jobs
    
    with Parallel(n_jobs=n_jobs, backend=backend) as parallel:
        fs = (delayed(integrate)(f, a + i * step, a + (i + 1) * step, n_iter=n_iter // n_jobs) for i in range(n_jobs))
    
    return sum(parallel(fs))

In [ ]:
%%timeit -n100
integrate_async_multiproc(math.cos, 0, math.pi / 2, n_iter=10**5, n_jobs=2)